In [2]:
from opal_ptx import CuModuleWrapper, kernel_transformer

from torch.utils.cpp_extension import load
import torch

from matplotlib import pyplot as pl


In [3]:

@kernel_transformer.kernel()
def mbarrier_test(buffer: "u64"):
    shMem: shared(u64, 1)
    
    #r1: u32 = 32
    ptx.mbarrier.init.shared.b64([shMem], 32)
    ptx.mbarrier.init.shared.b64 ([shMem], 1);

    completed: pred = 0 
    while completed == 0:
        ptx.mbarrier.try_wait.shared.b64(completed, [shMem], 1)
    
    buffer_global: u64
    ptx.cvta.to._global.u64(buffer_global, buffer)
    ptx.st._global.f32([buffer_global], 1.0)      


In [ ]:
kernel_builder = kernel_transformer.KernelBuilder()
mbarrier_test(kernel_builder)
kernel_code = kernel_builder.generate()

width, height = 1024, 1024

result = torch.zeros(32, dtype=torch.float32, device="cuda")

wrapper = CuModuleWrapper()
wrapper.load_ptx_code(kernel_code)

wrapper.launch_kernel("mandelbrot", (1, 1, 1), (32, 1, 1), (result.data_ptr(), width, height))
